You are given two short audio recordings.
 Audio 1 is your own voice saying the word “hello”.
 Audio 2 is your friend’s voice saying the word “hello”.
Your task is to compare these two signals using Dynamic Time Warping.
Tasks for the lab
Record your own voice saying “hello” and store it as Signal 1.


Record your friend saying “hello” and store it as Signal 2.


Convert both audio files into numerical time series by extracting their waveform data.


Normalize both signals so they have comparable amplitude ranges.


Apply Dynamic Time Warping on the two signals.


Produce the alignment path and compute the total DTW distance.


Interpret the results. Explain whether the signals are similar, and describe how DTW helps match two audio patterns that do not align perfectly in time.

